https://colab.research.google.com/drive/1GF-DM1sDnmErwUquxEdl4OFfnUWcVu6G#scrollTo=QBCdRXdr2c0q

In [39]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

In [40]:
# Load datasets
non_hostile = pd.read_csv("./src/non_hostile.csv")  # Contains 'Text', 'Label'
hostile = pd.read_csv("./src/hostile.csv")         # Contains 'Text', 'Label', 'SubLabel'

# Add SubLabel column to non-hostile and fill with 'None'
non_hostile['SubLabel'] = "None"

# Combine both datasets
data = pd.concat([non_hostile, hostile]).reset_index(drop=True)

In [41]:
data['Label'] = data['Label'].replace({'Non-hostile': 0, 'Hostile': 1, 'hostile': 1})
data.tail()

C:\Users\Scarlett\AppData\Local\Temp\ipykernel_8860\395125357.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Label'] = data['Label'].replace({'Non-hostile': 0, 'Hostile': 1, 'hostile': 1})


,Text,Label,SubLabel
14674,Bjp ના પોલીસ ના નખરા નઈ સહન કરવાના વિરોધ ની જર...,1.0,Offensive
14675,BJP ના લૂંટારા નેતા ઓ ની સાચેજ હવે ફાટવા લાગી ...,1.0,Defamation
14676,Bjp ના લોકો પાસે દંડ માંગવાની તેવડ નથી ગુલામી ...,1.0,Offensive
14677,Bjp ને વોટ ના આપો હજી ખરાબ હાલત કરશે આ બધા \r\n,1.0,Offensive
14678,BJP સરકાર માં પોલીસ ને પોતાની માં બહેનો ની પણ ...,1.0,Defamation


In [42]:
# divide the data into training val and test

train , val , test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

c:\Users\Scarlett\Desktop\Project\FARM-Stack\AI-Tweet-Detector\ai-test\ai_env\lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [43]:
len (train), len(val), len(test)

(8807, 2936, 2936)

In [45]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('Label')
    dataframe = dataframe["Text"]
    ds = tf.data.Dataset.from_tensor_slices((dataframe, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [49]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-large-cased")

def preprocess_text(texts):
    encodings = tokenizer(
        texts,
        max_length=128,  # Set appropriate max length
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    print(encodings)
    return {
        "input_word_ids": encodings["input_ids"],
        "input_mask": encodings["attention_mask"],
        "input_type_ids": encodings.get("token_type_ids", tf.zeros_like(encodings["input_ids"]))
    }


In [58]:
# Clean data before passing
train = train.dropna()
val = val.dropna()
test = test.dropna()

# Ensure correct data types
train['Text'] = train['Text'].astype(str)
train['Label'] = train['Label'].astype(int)
train['SubLabel'] = train['SubLabel'].astype(str)

val['Text'] = val['Text'].astype(str)
val['Label'] = val['Label'].astype(int)
val['SubLabel'] = val['SubLabel'].astype(str)

test['Text'] = test['Text'].astype(str)
test['Label'] = test['Label'].astype(int)
test['SubLabel'] = test['SubLabel'].astype(str)

# Create TensorFlow datasets
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

# # Preprocess text data
# train_data = preprocess_text(train["Text"].tolist())
# val_data = preprocess_text(val["Text"].tolist())
# test_data = preprocess_text(test["Text"].tolist())

In [61]:

# hub_layer = hub.KerasLayer(
#     "https://www.kaggle.com/models/google/muril/TensorFlow2/large/1",
#     trainable=True)

# train_data

list(train_ds)[0][0]

# hub_layer(list(train_data)[0][0])

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'Non',
       b'\xe0\xaa\xb6\xe0\xaa\xac\xe0\xab\x8d\xe0\xaa\xa6 \xe0\xaa\xb6\xe0\xaa\xa3\xe0\xaa\x97\xe0\xaa\xbe\xe0\xaa\xb0\xe0\xab\x80 \xe0\xaa\xaa\xe0\xaa\xa3 \xe0\xaa\xa6\xe0\xab\x87,\n\xe0\xaa\x85\xe0\xaa\xa8\xe0\xab\x87 \xe0\xaa\xb8\xe0\xaa\xb3\xe0\xaa\x97\xe0\xaa\xbe\xe0\xaa\xb5\xe0\xab\x80 \xe0\xaa\xaa\xe0\xaa\xa3 \xe0\xaa\xa6\xe0\xab\x87 !!\n\n#\xe0\xaa\xa6\xe0\xab\x87\xe0\xaa\xb6\xe0\xaa\xaa\xe0\xab\x8d\xe0\xaa\xb0\xe0\xab\x87\xe0\xaa\xae\xe0\xab\x80',
       b'\xe0\xaa\x85\xe0\xaa\xae\xe0\xaa\xa6\xe0\xaa\xbe\xe0\xaa\xb5\xe0\xaa\xbe\xe0\xaa\xa6 \xe0\xaa\x95\xe0\xaa\xb0\xe0\xab\x8d\xe0\xaa\xab\xe0\xab\x8d\xe0\xaa\xaf\xe0\xab\x82: \xe0\xaa\xac\xe0\xaa\xaa\xe0\xab\x8b\xe0\xaa\xb0 \xe0\xaa\xb8\xe0\xab\x81\xe0\xaa\xa7\xe0\xab\x80\xe0\xaa\xae\xe0\xaa\xbe\xe0\xaa\x82 \xe0\xaa\x9c\xe0\xaa\xbe\xe0\xaa\xb9\xe0\xab\x87\xe0\xaa\xb0\xe0\xaa\xa8\xe0\xaa\xbe\xe0\xaa\xae\xe0\xaa\xbe \xe0\xaa\xad\xe0\xaa\x82\xe0\xaa\x97\xe0\xaa\xa8\xe0\xa